# Deep Learning

## Assignment 4

In [1]:
from __future__ import print_function
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from six.moves import cPickle as pickle
from six.moves import range

In [2]:
pickle_file = 'notMNIST.pickle'
with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save
    print('Training set', train_dataset.shape, train_labels.shape)
    print('Validation set', valid_dataset.shape, valid_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Convolutions need the image data formatted as a cube (width by height by #channels)

In [3]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

def reformat(dataset, labels):
    dataset = dataset.reshape((-1, image_size, image_size, num_channels)).astype(np.float32)
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
    return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [4]:
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1)) / predictions.shape[0])

In [5]:
tf_config = tf.ConfigProto()
tf_config.gpu_options.allow_growth = True
tf_config.log_device_placement = True

### Two convolutional layers

In [6]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():
    #Input data.
    tf_train_dataset = tf.placeholder(tf.float32, shape = (batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape = (batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    #Variables.
    layer_1_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth], stddev = 0.1))
    layer_1_biases = tf.Variable(tf.zeros([depth]))
    layer_2_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth, depth], stddev = 0.1))
    layer_2_biases = tf.Variable(tf.constant(1.0, shape = [depth]))
    layer_3_weights = tf.Variable(tf.truncated_normal([image_size // 4 * image_size // 4 * depth, num_hidden], \
                                                      stddev = 0.1))
    layer_3_biases = tf.Variable(tf.constant(1.0, shape = [num_hidden]))
    layer_4_weights = tf.Variable(tf.truncated_normal([num_hidden, num_labels], stddev = 0.1))
    layer_4_biases = tf.Variable(tf.constant(1.0, shape = [num_labels]))
    
    #Model.
    def model(data):
        conv = tf.nn.conv2d(data, layer_1_weights, [1, 2, 2, 1], padding = 'SAME')
        hidden = tf.nn.relu(conv + layer_1_biases)
        conv = tf.nn.conv2d(hidden, layer_2_weights, [1, 2, 2, 1], padding = 'SAME')
        hidden = tf.nn.relu(conv + layer_2_biases)
        shape = hidden.get_shape().as_list()
        reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden = tf.nn.relu(tf.matmul(reshape, layer_3_weights) + layer_3_biases)
        return tf.matmul(hidden, layer_4_weights) + layer_4_biases
    
    #Training computations.
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels = tf_train_labels, logits = logits))
    
    #Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
    
    #Predictions for training, validation, testing data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))   

In [7]:
num_steps = 3001
with tf.Session(graph = graph, config = tf_config) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset: batch_data, tf_train_labels: batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict = feed_dict)
        if (step % 50 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))


Initialized
Minibatch loss at step 0: 3.282560
Minibatch accuracy: 6.2%
Validation accuracy: 10.1%
Minibatch loss at step 50: 1.887189
Minibatch accuracy: 31.2%
Validation accuracy: 43.6%
Minibatch loss at step 100: 0.865850
Minibatch accuracy: 56.2%
Validation accuracy: 71.1%
Minibatch loss at step 150: 1.237040
Minibatch accuracy: 68.8%
Validation accuracy: 76.3%
Minibatch loss at step 200: 0.919133
Minibatch accuracy: 75.0%
Validation accuracy: 77.4%
Minibatch loss at step 250: 0.677687
Minibatch accuracy: 75.0%
Validation accuracy: 79.8%
Minibatch loss at step 300: 0.798244
Minibatch accuracy: 75.0%
Validation accuracy: 80.0%
Minibatch loss at step 350: 0.676490
Minibatch accuracy: 81.2%
Validation accuracy: 79.8%
Minibatch loss at step 400: 1.107863
Minibatch accuracy: 62.5%
Validation accuracy: 78.4%
Minibatch loss at step 450: 0.421390
Minibatch accuracy: 93.8%
Validation accuracy: 79.4%
Minibatch loss at step 500: 1.273224
Minibatch accuracy: 56.2%
Validation accuracy: 81.4%
Mi

## Problem 1

In [8]:
batch_size = 128
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()
with graph.as_default():
    #Input data.
    tf_train_dataset = tf.placeholder(tf.float32, shape = (batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape = (batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    #Variables.
    layer_1_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth], stddev = 0.1))
    layer_1_biases = tf.Variable(tf.zeros([depth]))
    layer_2_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth, depth], stddev = 0.1))
    layer_2_biases = tf.Variable(tf.constant(1.0, shape = [depth]))
    layer_3_weights = tf.Variable(tf.truncated_normal([image_size // 4 * image_size // 4 * depth, num_hidden], \
                                                      stddev = 0.1))
    layer_3_biases = tf.Variable(tf.constant(1.0, shape = [num_hidden]))
    layer_4_weights = tf.Variable(tf.truncated_normal([num_hidden, num_labels], stddev = 0.1))
    layer_4_biases = tf.Variable(tf.constant(1.0, shape = [num_labels]))
    
    #Model.
    def model(data):
        conv_1 = tf.nn.conv2d(data, layer_1_weights, [1, 1, 1, 1], padding = 'SAME')
        hidden_1 = tf.nn.relu(conv_1 + layer_1_biases)
        pool_1 = tf.nn.max_pool(hidden_1, [1, 2, 2, 1], [1, 2, 2, 1], padding = 'SAME')
        conv_2 = tf.nn.conv2d(pool_1, layer_2_weights, [1, 1, 1, 1], padding = 'SAME')
        hidden_2 = tf.nn.relu(conv_2 + layer_2_biases)
        pool_2 = tf.nn.max_pool(hidden_2, [1, 2, 2, 1], [1, 2, 2, 1], padding = 'SAME')
        shape = pool_2.get_shape().as_list()
        reshape = tf.reshape(pool_2, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden = tf.nn.relu(tf.matmul(reshape, layer_3_weights) + layer_3_biases)
        return tf.matmul(hidden, layer_4_weights) + layer_4_biases
    
    #Training computations.
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels = tf_train_labels, logits = logits))
    
    #Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
    
    #Predictions for training, validation, testing data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))   

In [9]:
num_steps = 3001

with tf.Session(graph = graph, config = tf_config) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset: batch_data, tf_train_labels: batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict = feed_dict)
        if (step % 100 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))


Initialized
Minibatch loss at step 0: 3.037343
Minibatch accuracy: 13.3%
Validation accuracy: 11.3%
Minibatch loss at step 100: 0.856928
Minibatch accuracy: 74.2%
Validation accuracy: 79.0%
Minibatch loss at step 200: 0.690425
Minibatch accuracy: 82.0%
Validation accuracy: 80.0%
Minibatch loss at step 300: 0.495563
Minibatch accuracy: 85.2%
Validation accuracy: 82.6%
Minibatch loss at step 400: 0.531808
Minibatch accuracy: 83.6%
Validation accuracy: 83.5%
Minibatch loss at step 500: 0.634934
Minibatch accuracy: 83.6%
Validation accuracy: 84.3%
Minibatch loss at step 600: 0.546333
Minibatch accuracy: 80.5%
Validation accuracy: 84.5%
Minibatch loss at step 700: 0.634372
Minibatch accuracy: 81.2%
Validation accuracy: 85.1%
Minibatch loss at step 800: 0.452586
Minibatch accuracy: 84.4%
Validation accuracy: 85.1%
Minibatch loss at step 900: 0.320847
Minibatch accuracy: 92.2%
Validation accuracy: 85.8%
Minibatch loss at step 1000: 0.501428
Minibatch accuracy: 85.2%
Validation accuracy: 85.5%

## Problem 2

In [33]:
batch_size = 128
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():
    #Input data.
    tf_train_dataset = tf.placeholder(tf.float32, shape = (batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape = (batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    #Variables.
    layer_1_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth], stddev = 0.1))
    layer_1_biases = tf.Variable(tf.zeros([depth]))
    layer_2_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth, depth]))
    layer_2_biases = tf.Variable(tf.constant(1.0, shape = [depth]))
    size_3 = 4
    layer_3_weights = tf.Variable(tf.truncated_normal([size_3 * size_3 * depth, num_hidden], stddev = 0.1))
    layer_3_biases = tf.Variable(tf.constant(1.0, shape = [num_hidden]))
    layer_4_weights = tf.Variable(tf.truncated_normal([num_hidden, num_labels], stddev = 0.1))
    layer_4_biases = tf.Variable(tf.constant(1.0, shape = [num_labels]))
    
    #Model.
    def model(data):
         #C1 input size: 28x28
        conv_1 = tf.nn.conv2d(data, layer_1_weights, [1, 1, 1, 1], padding = 'VALID')
        layer_1 = tf.nn.relu(conv_1 + layer_1_biases)
        #S2 input size: 24x24
        layer_2 = tf.nn.avg_pool(layer_1, [1, 2, 2, 1], [1, 2, 2, 1], padding = 'VALID')
        #C3 input size: 12x12
        conv_3 = tf.nn.conv2d(layer_2, layer_2_weights, [1, 1, 1, 1], padding = 'VALID')
        layer_3 = tf.nn.relu(conv_3 + layer_2_biases)
        #S4 input size: 8x8
        layer_4 = tf.nn.avg_pool(layer_3, [1, 2, 2, 1], [1, 2, 2, 1], padding = 'VALID')
        #F6 input size: 4x4
        shape = layer_4.get_shape().as_list()
        reshape = tf.reshape(layer_4, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden = tf.nn.relu(tf.matmul(reshape, layer_3_weights) + layer_3_biases)
        return tf.matmul(hidden, layer_4_weights) + layer_4_biases
    
     #Training computation.
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels = tf_train_labels, logits = logits))
    
    #Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
    
    #Predictions for training, validation, testing data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [34]:
num_steps = 20001

with tf.Session(graph = graph, config = tf_config) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset: batch_data, tf_train_labels: batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict = feed_dict)
        if (step % 500 == 0):
            print(offset)
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
0
Minibatch loss at step 0: 3.510392
Minibatch accuracy: 6.2%
Validation accuracy: 14.0%
64000
Minibatch loss at step 500: 0.643637
Minibatch accuracy: 82.0%
Validation accuracy: 83.0%
128000
Minibatch loss at step 1000: 0.516864
Minibatch accuracy: 85.2%
Validation accuracy: 85.0%
192000
Minibatch loss at step 1500: 0.442737
Minibatch accuracy: 88.3%
Validation accuracy: 85.6%
56128
Minibatch loss at step 2000: 0.522525
Minibatch accuracy: 86.7%
Validation accuracy: 86.2%
120128
Minibatch loss at step 2500: 0.483779
Minibatch accuracy: 86.7%
Validation accuracy: 86.5%
184128
Minibatch loss at step 3000: 0.446386
Minibatch accuracy: 87.5%
Validation accuracy: 86.5%
48256
Minibatch loss at step 3500: 0.360615
Minibatch accuracy: 89.8%
Validation accuracy: 87.1%
112256
Minibatch loss at step 4000: 0.351253
Minibatch accuracy: 89.8%
Validation accuracy: 87.4%
176256
Minibatch loss at step 4500: 0.629979
Minibatch accuracy: 81.2%
Validation accuracy: 87.6%
40384
Minibatch loss 